In [1]:
!pip install --upgrade pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [2]:
#Descargamos la libreria de transformers que usaremos para descargar el modelo como su respectivo tokenizador
!pip  install transformers --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
!pip install Datasets --quiet
from datasets import DatasetDict, Dataset
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet
!pip install transformers[torch]
#Installamos esta librería para el preprocesamiento necesario que requieré el modelo RoBertTuito:
!pip install pysentimiento --quiet
from pysentimiento.preprocessing import preprocess_tweet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.6 MB/s eta 0:00:00


In [3]:
# Importamos las dependencias necesarias :
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from google.colab import drive  #Montador de drive
from sklearn.model_selection import train_test_split
import gc

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# 2. Importamos el conjunto de datos que vamos a usar para nuestro problema

In [4]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Datasets a limpio /EXIST 2021 dataset_esp.csv')

ValueError: mount failed

In [ ]:
df.head()

In [ ]:
# 1. Cambiamos el nombre de las columnas e eliminamos aquellas que no necesitamos:
columns_to_remove = ['test_case', 'id', 'source','language','task2']
df = df.rename(columns = {"task1": "label"}).drop(columns=columns_to_remove, axis=1)
df.head()

In [ ]:
# 2. Cambiamos los valores nominales de nuestra etiqueta por valores númericos
df['label'] = df['label'].replace(['non-sexist','sexist'],[0, 1])
df.head()

In [ ]:
# 3. Eliminamos cualquier fila que haya podido quedar en blanco:
df = df.dropna()

In [ ]:
from pysentimiento.preprocessing import preprocess_tweet

In [ ]:
df['text'].apply(preprocess_tweet)

In [ ]:
df.head()

In [ ]:
# 4. Dividiremos el dataset en el 80% para el entrenamiento, el 10% para el proceso de validation, y un 10% para testear los resultados del modelo.
train_df, valtest_df = train_test_split(df, test_size = 0.2, random_state = 42)
val_df, test_df = train_test_split(valtest_df, test_size = 0.5, random_state = 42)
train_df.shape, val_df.shape, test_df.shape

In [ ]:
train = Dataset.from_pandas(train_df)
validation = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

In [ ]:
train



```
# Funciones para entrenar el modelo
```



In [ ]:
# Define a function to train a model v2
class TrainModel:
    """

    Attributes:
      tokenizer:
      modelo:
    """

    def __init__(self, Modelo):
        # Check if execution will be performed on cuda:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(Modelo)
        self.modelo = AutoModelForSequenceClassification.from_pretrained(Modelo, num_labels=2).to(device)

    def tokenizador(self, batch):
        return self.tokenizer(batch["text"], padding=True, max_length=128, truncation=True)

    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        f1 = f1_score(labels, preds, average="weighted")
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc, "f1": f1}

    def train(self, train, val):
        # Prepare the dataset:
        Tweets_Dataset = DatasetDict({'train': train, 'val': val, 'test': test})

        # Apply tokenization to the entire dataset using the map function:
        #Tweets_Dataset = Tweets_Dataset.remove_columns(["__index_level_0__"])
        Tweets_Encoded = Tweets_Dataset.map(self.tokenizador, batched=True, batch_size=None)


        # Ensure objects are of torch type
        Tweets_Encoded.set_format("torch", columns=["label", "input_ids", "attention_mask"])

        # Definimos los hiperparametros de cada modelo
        if self.modelo == 'sdadas/xlm-roberta-large-twitter':
            learning_rate = 2e-5
            per_device_train_batch_size = 16
            num_train_epochs=2
        elif self.modelo == 'Twitter/twhin-bert-base':
            learning_rate = 2e-5
            per_device_train_batch_size = 16
            num_train_epochs=4
        else:
            learning_rate = 1e-5
            per_device_train_batch_size = 16
            num_train_epochs=3

        training_args = TrainingArguments(
              output_dir='./results',
              num_train_epochs= num_train_epochs,
              learning_rate = learning_rate,
              per_device_train_batch_size= per_device_train_batch_size,
              per_device_eval_batch_size=8,
              warmup_steps=500,
              weight_decay=0.01,
              evaluation_strategy="epoch",
              logging_dir='./logs',
              )

        # Create the Trainer and train the model
        trainer = Trainer(
            model=self.modelo,
            args=training_args,
            compute_metrics=self.compute_metrics,
            train_dataset=Tweets_Encoded['train'],
            eval_dataset=Tweets_Encoded['val'],
        )
        trainer.train()

        return self.modelo, self.tokenizer

In [ ]:
train = train.remove_columns(["__index_level_0__"])
validation = validation.remove_columns(["__index_level_0__"])
test = test.remove_columns(["__index_level_0__"])

In [ ]:
train = Dataset.from_pandas(train_df)
validation = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

In [ ]:
sdadas_trainer = TrainModel('sdadas/xlm-roberta-large-twitter')
twhin_trainer = TrainModel('Twitter/twhin-bert-base')
robertuito_trainer = TrainModel('pysentimiento/robertuito-base-uncased')

In [ ]:
Modelo1, tokenizador1 = sdadas_trainer.train(train, validation)
Modelo2, tokenizador2 = twhin_trainer.train(train, validation)
Modelo3, tokenizador3 = robertuito_trainer.train(train, validation)

In [ ]:
modelos = [Modelo1,Modelo2,Modelo3]
tokenizadores = [tokenizador1,tokenizador2,tokenizador3]

In [ ]:
https://www.kaggle.com/code/emiz6413/transformers-united-majority-voting/notebook

In [ ]:
def majority_voting(models, tokenizers, texts):
    # Initialize a list to store the majority vote for each text
    majority_votes = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for text in texts:
        # Get the predictions from each model
        predictions = []
        for model, tokenizer in zip(models, tokenizers):
            inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=128, truncation=True)
            inputs = {name: tensor.to(device) for name, tensor in inputs.items()}

            with torch.no_grad():
                model_logits = model(**inputs).logits
                _, preds = torch.max(model_logits, dim=1)
                predictions.append(preds)

        # majority voting
        predictions = torch.stack(predictions).to(device)
        majority_vote, _ = torch.mode(predictions, dim=0)

        # Append the majority vote to the list
        majority_votes.append(majority_vote.item())

    return majority_votes

In [ ]:
predictions = majority_voting(modelos, tokenizadores, validation['text'])

In [ ]:
def compute_metrics(true_labels, predictions):
    # Compute the accuracy
    accuracy = accuracy_score(true_labels, predictions)

    # Compute the F1 score
    f1 = f1_score(true_labels, predictions, average='weighted')

    return accuracy, f1

In [ ]:
accuracy , f1 = compute_metrics(validation['label'],predictions)

In [ ]:
accuracy

In [ ]:
f1

In [ ]:
 Define a function to train a model v2
class TrainModel:
    """

    Attributes:
      tokenizer:
      modelo:
    """

    def __init__(self, Modelo):
        # Check if execution will be performed on cuda:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(Modelo)
        self.modelo = AutoModelForSequenceClassification.from_pretrained(Modelo, num_labels=2).to(device)

    def tokenizador(self, batch):
        return self.tokenizer(batch["text"], padding=True, max_length=128, truncation=True)

    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        f1 = f1_score(labels, preds, average="weighted")
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc, "f1": f1}

    def train(self, train, val):
        # Prepare the dataset:
        Tweets_Dataset = DatasetDict({'train': train, 'val': val, 'test': test})

        # Apply tokenization to the entire dataset using the map function:
        Tweets_Dataset = Tweets_Dataset.remove_columns(["__index_level_0__"])
        Tweets_Encoded = Tweets_Dataset.map(self.tokenizador, batched=True, batch_size=None)

        # Ensure objects are of torch type
        Tweets_Encoded.set_format("torch", columns=["label", "input_ids", "attention_mask"])

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=2,
            learning_rate = 2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            warmup_steps=500,
            weight_decay=0.01,
            evaluation_strategy="epoch",
            logging_dir='./logs',
        )

        # Create the Trainer and train the model
        trainer = Trainer(
            model=self.modelo,
            args=training_args,
            compute_metrics=self.compute_metrics,
            train_dataset=Tweets_Encoded['train'],
            eval_dataset=Tweets_Encoded['val'],
        )
        trainer.train()

        return self.modelo, self.tokenizer